In [9]:
import numpy as np      
from scipy import stats
from scipy.fft import fft,fftfreq# Matlab like syntax for linear algebra and functions
from numpy.linalg import matrix_power                  # This is to raise matrices to powers
import matplotlib.pyplot as plt                        # Plots and figures like you know them from Matlab
from scipy.signal import savgol_filter



In [10]:
########## Import data ############
fname = 'Nutlin_1uM.txt'
DD = []
with open( fname, 'r' ) as f :
    for line in f:
        line = line.strip()        
        columns = line.split(',')
        ddtmp = []
        for i in range(len(columns)):
            ddtmp.append(float(columns[i]))
        DD.append(ddtmp)
DD = np.array(DD)
LC = len(DD[:,0])
NC = len(DD[0,:])
print(LC,NC)
t = np.linspace(1,NC,NC)
t/=6.

MDx = np.mean(DD,axis=0)
SDx = np.std(DD,axis=0)


222 169


In [12]:

npol = 3
Fs = 1/6.  # Sampling rate
T = 1/Fs  # Sampling interval
Peaks = []
Vals = []
for i in range(LC):
    x = DD[i,:]
    x /= x[0]
    
    fxp = np.polyfit(t[36:],x[36:],1)
    Vals.append([fxp[0],fxp[1]])
    
    fxp = np.polyfit(t[:36],x[:36],2)
    me = np.mean(x[100:])
    Peaks.append([fxp[0],fxp[1],fxp[2]])

Peaks = np.array(Peaks)

yax = -Peaks[:,1]/(2*Peaks[:,0])

idx_good = np.arange(len(yax))
idx_bad = []

Zval_yax = (yax-np.mean(yax))/np.std(yax)
Prob = 1/np.sqrt(2*np.pi)*np.exp(-0.5*(Zval_yax)**2)
CCrit = Prob*len(Zval_yax)
############ Applying Chauvenets criterion to the position of the first pulse
yax_new = yax.copy()
while (np.min(CCrit) < 0.5):
    
    idx = np.argmin(CCrit)
    idx_bad.append(idx)
    idx_good = np.delete(idx_good, idx)
    
    Zval_yax = (yax[idx_good]-np.mean(yax[idx_good]))/np.std(yax[idx_good])
    Prob = 1/np.sqrt(2*np.pi)*np.exp(-0.5*(Zval_yax)**2)
    CCrit = Prob*len(Zval_yax)

    
PlotTrue = False
if PlotTrue == True:
    plt.hist(yax[idx_good],20,edgecolor = 'k',facecolor = [.2,.5,.9],lw=3)
    plt.show()

############ Applying Chauvenets criterion to slope of dynamics

Vals = np.array(Vals)
if PlotTrue == True:
    plt.hist(Vals[idx_good,0],20,edgecolor = 'k',facecolor = [0.5,0,0.2],lw=3)
    plt.show()

yder = Vals[:,0]
Zval_yax = (yder[idx_good]-np.mean(yder[idx_good]))/np.std(yder[idx_good])
Prob = 1/np.sqrt(2*np.pi)*np.exp(-0.5*(Zval_yax)**2)
CCrit = Prob*len(Zval_yax)

while (np.min(CCrit) < 0.5):
    print(len(Zval_yax))
    idx = np.argmin(CCrit)
    idx_bad.append(idx_good[idx])
    idx_good = np.delete(idx_good, idx)
    
    Zval_yax = (yder[idx_good]-np.mean(yder[idx_good]))/np.std(yder[idx_good])
    Prob = 1/np.sqrt(2*np.pi)*np.exp(-0.5*(Zval_yax)**2)
    CCrit = Prob*len(Zval_yax)
if PlotTrue == True: 
    plt.hist(Vals[idx_good,0],20,edgecolor = 'k',facecolor = [.8,.1,0.1],lw=3)
    plt.show()

############ Now "idx_good" refer to the traces accepted by the method


176
175
174
173
172
171
170


# Plot single traces

In [13]:
from scipy import optimize
from matplotlib.pyplot import figure

npol = 5


for i in idx_good:
    print(i)
    x = DD[i,:]
    x /= x[0]
    
    tstart = 0
    tend = 170
    tf = t[tstart:tend]-t[tstart];
    fxn = np.polyfit(tf,x[tstart:tend],npol)
    
    s = 0
    for ffi in range(len(fxn)):
        s += fxn[ffi]*tf**(npol-ffi)
    xn = x[tstart:tend]-s
    
    x2 = savgol_filter(xn, 13, 2)
    
    
    #params_decay, params_covariance_decay = optimize.curve_fit(test_func_decay, tf, xn,bounds=([0,2*np.pi/10,0,0], [10, 2*np.pi/1, 2*np.pi,.5]),
    #                p0=[np.max(x2), 2*np.pi/Permax,np.pi,0.1])
    


    
    #plt.figure(figsize=(12, 8))
    #plt.plot(t,xn,'b',lw=5)
    if PlotTrue == True:
        plt.plot(t,x2,'g',lw=5)
        plt.show()
        
    

0
2
3
4
5
7
9
10
11
12
13
17
18
20
21
22
23
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
49
50
51
52
53
54
55
56
57
58
59
60
61
62
64
66
68
72
73
74
76
77
78
80
81
82
83
84
85
88
89
90
91
92
93
94
95
98
101
102
103
104
105
106
107
108
109
110
111
112
113
115
116
117
118
119
120
121
122
124
127
130
131
132
133
135
136
138
140
142
144
146
147
148
149
150
151
156
158
159
161
162
163
164
165
166
167
171
172
173
174
175
176
178
179
180
181
183
184
185
187
188
189
190
193
194
195
198
199
200
201
202
204
205
206
207
208
209
210
211
212
214
215
216
217
218
219
220
221


# Fit to exponential curve

In [14]:
from scipy import optimize
from matplotlib.pyplot import figure


def test_func(x, a, b,c):
    return a *np.sin(b * x + c)

def test_func_decay(x, a, b,c,d):
    return a *np.exp(-d*x)*np.sin(b * x + c)
Pars = []

npol = 5


Fs = 6.  # Sampling rate
T = 1/Fs  # Sampling interval
Peaks = []
MAGS = []
Pers = []
c = 0
DD_good = []
Decay = []

for i in idx_good:
    x = DD[i,:]
    x /= x[0]
    
    tstart = 0
    tend = 170
    tf = t[tstart:tend]-t[tstart];
    fxn = np.polyfit(tf,x[tstart:tend],npol)
    
    s = 0
    for ffi in range(len(fxn)):
        s += fxn[ffi]*tf**(npol-ffi)
    xn = x[tstart:tend]-s
    #plt.plot(t[36:],s,'--r',lw=3) 
    x2 = savgol_filter(xn, 15, 2)
    #fft_result = np.fft.fft(x2)
    fft_result = np.fft.fft(xn)
    N = len(fft_result)
    frequencies = np.fft.fftfreq(N, T)     
    half = int(N/2)
    freq = frequencies[:half]
    # Compute the magnitude of the FFT
    magnitude = np.abs(fft_result);       
    magx = magnitude[:half]
    fmin = freq[np.argmax(magx)]
    Permax = 1/(fmin)
    if (Permax>10):
        Permax = 5.5
   
    
    params, params_covariance = optimize.curve_fit(test_func, tf, xn,
                                               p0=[np.max(x2), 2*np.pi/5.5,np.pi])
    param_bounds=([-np.inf,0,-np.inf],[np.inf,2,np.inf])
    params_decay, params_covariance_decay = optimize.curve_fit(test_func_decay, tf, xn,bounds=([0,2*np.pi/10,0,0], [10, 2*np.pi/1, 2*np.pi,.5]),
                                               p0=[np.max(x2), 2*np.pi/5.5,np.pi,0.1])
    if (np.max(x[:36])>0.5*np.max(x[36:])):
        Pars.append([params_decay[3],2*np.pi/params_decay[1]])
        if PlotTrue == True:
            plt.figure(figsize=(15, 8))
            plt.plot(t,x,'g',lw=5,alpha=0.2)
            plt.plot(t,xn,'b',lw=5)
            plt.plot(tf,test_func_decay(tf,params_decay[0],params_decay[1],params_decay[2],params_decay[3]),color = [.8,0,0.8],lw=5)
            plt.show()
        
        c+=1
    else:
        if PlotTrue == True:
            plt.plot(tf,xn,'r')
            plt.show()
Pers = np.array(Pers)
if PlotTrue == True:
    plt.hist(Pers,20,edgecolor = 'k',facecolor = [0.6,.8,0.1],lw=3)
    plt.show()

print(len(idx_good))

Pars = np.array(Pars)
if PlotTrue == True:
    plt.hist((Pars[:,0]), 14,edgecolor = 'k',facecolor = [0.8,.0,0.8],lw=3)
    plt.show()
    
print(np.mean(Pars[:,0]),np.median(Pars[:,0]))
#plt.xscale("log")
print(np.mean(Pars[:,0]),np.median(Pars[:,0]))
if PlotTrue == True:
    plt.hist((Pars[:,1]), 14,edgecolor = 'k',facecolor = [1,.2,0.5],lw=3)
    plt.show()

169
0.1055957858703741 0.08989037270569947
0.1055957858703741 0.08989037270569947


## Plot FFT spectrum

In [15]:
from scipy import optimize
from matplotlib.pyplot import figure


def test_func(x, a, b,c):
    return a *np.sin(b * x + c)

def test_func_decay(x, a, b,c,d):
    return a *np.exp(-d*x)*np.sin(b * x + c)
Pars = []

npol = 5


Fs = 6.  # Sampling rate
T = 1/Fs  # Sampling interval
Peaks = []
MAGS = []
Pers = []
c = 0
DD_good = []
Decay = []
for i in idx_good:
    x = DD[i,:]
    x /= x[0]
    
    tstart = 0
    tend = 170
    tf = t[tstart:tend]-t[tstart];
    fxn = np.polyfit(tf,x[tstart:tend],npol)
    
    s = 0
    for ffi in range(len(fxn)):
        s += fxn[ffi]*tf**(npol-ffi)
    xn = x[tstart:tend]-s

    fft_result = np.fft.fft(xn)
    N = len(fft_result)
    frequencies = np.fft.fftfreq(N, T)     
    half = int(N/2)
    freq = frequencies[:half]
    # Compute the magnitude of the FFT
    magnitude = np.abs(fft_result);       
    magx = magnitude[:half]
    fmin = freq[np.argmax(magx)]
    Permax = 1/(fmin)
    if (Permax>10):
        Permax = 5.5
   
    
    params, params_covariance = optimize.curve_fit(test_func, tf[:36], xn[:36],
                                               p0=[np.max(x2), 2*np.pi/5.5,np.pi])
    osci_0 = params[0]
    osci_1 = params[1]
    osci_2 = params[2]
    param_bounds=([-np.inf,0,-np.inf],[np.inf,2,np.inf])
    params_decay, params_covariance_decay = optimize.curve_fit(test_func_decay, tf, xn,bounds=([0,2*np.pi/10,0,0], [10, 2*np.pi/1, 2*np.pi,.5]),
                                               p0=[np.max(x2), 2*np.pi/Permax,np.pi,0.1])
    if (np.max(x[:36])>0.5*np.max(x[36:])):
        Pars.append([params_decay[3],2*np.pi/params_decay[1]])
        
        if PlotTrue == True:
            plt.figure(figsize=(15, 8))
            plt.plot(t,x,'g',lw=5,alpha=0.2)
            plt.plot(t,xn,'b',lw=5)
            plt.plot(t[40:],osci_0/2*np.sin(2*np.pi/Permax*t[40:] + 1*np.pi),color = [1,.6,.4],lw=5)        
            plt.show()
        
            fplot = frequencies[:half]; mplot = magnitude[:half]
            plt.figure(figsize=(15, 8))
            plt.plot(fplot[:-50],mplot[:-50],'g',lw=5,alpha=1);
            plt.plot(1/5.5*np.ones(10),np.linspace(0,np.max(magnitude[:half]),10),'--k',lw=5);
            plt.title("FFT Spectrum"); plt.xlabel("Frequency (Hz)"); plt.ylabel("Magnitude"); #plt.grid()
            plt.show()
        c+=1
    else:
        if PlotTrue == True:
            plt.plot(tf,xn,'r')
            plt.show()
    

Pars = np.array(Pars)
if PlotTrue == True:
    plt.hist((Pars[:,0]), 14,edgecolor = 'k',facecolor = [0.8,.0,0.8],lw=3)
    plt.show()
print(np.mean(Pars[:,0]),np.median(Pars[:,0]))
#plt.xscale("log")

print(np.mean(Pars[:,1]),np.median(Pars[:,1]))
if PlotTrue == True:
    plt.hist((Pars[:,1]), 14,edgecolor = 'k',facecolor = [1,.2,0.5],lw=3)
    plt.show()

0.11447604607450713 0.10675090670350507
6.122906364460982 5.670598827212221
